In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
from helpers.hull_moving_average_concavity import calculate_hma_signals
from helpers.smoothed_heikin_ashi import calculate_smoothed_heikin_ashi
from helpers.apply_labels import apply_labels

In [ ]:
data = yf.download('QQQ', start='2000-01-01').reset_index()

data = calculate_smoothed_heikin_ashi(data)
data = calculate_hma_signals(data)
data['EMA_50'] = data['Close'].ewm(span=50, adjust=False).mean()
data['EMA_200'] = data['Close'].ewm(span=200, adjust=False).mean()
data = apply_labels(data)


In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

# Merge the buy and sell signals into a single DataFrame by matching each buy with the next sell
trades = pd.DataFrame(columns=['Buy Date', 'Sell Date', 'Buy Price', 'Sell Price'])

# Track the index of the last sell to avoid matching previous sells
last_sell_index = 0

buy_signals = data[data['Alert'] == 'Long Alert']
sell_signals = data[data['Alert'] == 'Sell Alert']

for index, buy in buy_signals.iterrows():
    # Find the next sell signal after the buy
    next_sell = sell_signals[sell_signals['Date'] > buy['Date']]
    if not next_sell.empty:
        next_sell = next_sell.iloc[0]  # Take the first sell after the buy
        if next_sell.name > last_sell_index:  # Check if it's a new sell signal
            last_sell_index = next_sell.name
            # Record the trade
            new_row = {
                'Buy Date': buy['Date'],
                'Sell Date': next_sell['Date'],
                'Buy Price': buy['Close'],
                'Sell Price': next_sell['Close'],
                'Profit': (next_sell['Close'] - buy['Close'])
            }
            new_row_df = pd.DataFrame([new_row])
            trades = pd.concat([trades, new_row_df], ignore_index=True)

# Calculate the % gain/loss for each trade
trades['Return'] = (trades['Sell Price'] / trades['Buy Price'] - 1) * 100

In [ ]:
investment_period_days = (trades['Sell Date'].max() - trades['Buy Date'].min()).days
investment_period_years = investment_period_days / 365.25  # Average days in a year accounting for leap year

initial_value = trades['Buy Price'].iloc[0]

final_value = initial_value + trades['Profit'].sum()

# Calculate CAGR
cagr = (final_value / initial_value) ** (1 / investment_period_years) - 1

print(f"CAGR: {cagr * 100:.2f}%")

In [ ]:
buy_hold_entry = data.iloc[0]['Close']  # First available close price
buy_hold_exit = data.iloc[-1]['Close']  # Last available close price

# Calculate the total return for Buy and Hold
buy_hold_profit = buy_hold_exit - buy_hold_entry
buy_hold_return = (buy_hold_exit / buy_hold_entry - 1) * 100

buy_hold_entry, buy_hold_exit, buy_hold_profit, buy_hold_return

In [ ]:
investment_period_years = (data['Date'].iloc[-1] - data['Date'].iloc[0]).days / 365.25

# CAGR for both strategies
cagr_trading = ((trades['Sell Price'].iloc[-1] / trades['Buy Price'].iloc[0]) ** (1 / investment_period_years)) - 1
cagr_buy_hold = ((buy_hold_exit / buy_hold_entry) ** (1 / investment_period_years)) - 1

# Calculate Sharpe Ratio
# Assuming risk-free rate as 0.25% annually
risk_free_rate = 0.0025
trading_returns = trades['Return'] / 100
average_trading_return = trading_returns.mean()
std_dev_trading = trading_returns.std()

# Calculate Sharpe Ratio for strategy
sharpe_ratio_trading = (average_trading_return - risk_free_rate) / std_dev_trading

# Calculate Sharpe Ratio for Buy and Hold
buy_hold_return_decimal = buy_hold_return / 100
sharpe_ratio_buy_hold = (buy_hold_return_decimal / investment_period_years - risk_free_rate) / (buy_hold_return_decimal / np.sqrt(investment_period_years))

cagr_trading, cagr_buy_hold, sharpe_ratio_trading, sharpe_ratio_buy_hold